<a href="https://colab.research.google.com/github/ipaguilarc/-Clase_ML_2025_4/blob/main/notebooks/NB_13_2%20SELECCION%20GRID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maestría en Maestría en Ciencia de Datos e Inteligencia Artificial
#### 8. Machine Learning and Deep Learning
#### Docente: Msc. Renzo Claure Aracena.

## SELECCION DE MODELOS
### Validacion cruzada

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [8]:
base = pd.read_csv('cancer.csv', delimiter = ';', decimal=',')

In [9]:
base.head()

,ID,Tipo,Radio,Textura,Perimetro,Area,Suavidad,Compactavidad,Concavidad,Puntos_concavos,Simetria,Fractal
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [10]:
y = base['Tipo']
y.value_counts()

,count
Tipo,
B,357
M,212


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [12]:
X = base.drop(['ID', 'Tipo'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=0)

In [13]:
clasif = SVC(kernel='linear', C=1)
clasif.fit(X_train, y_train)
clasif.score(X_test, y_test)

0.9090909090909091

In [14]:
print('Validacion Cruzada Entrenamiento (exactitud/acierto):', cross_val_score(clasif, X_train, y_train, cv=5))

Validacion Cruzada Entrenamiento (exactitud/acierto): [0.93023256 0.90588235 0.84705882 0.95294118 0.94117647]


In [15]:
cv_score = cross_val_score(clasif, X_train, y_train, cv=5)
cv_score.mean()

np.float64(0.9154582763337894)

In [16]:
from sklearn.metrics import roc_auc_score, f1_score

In [17]:
y_pred = clasif.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred)
print(f'El indicador AUC sobre la base de comprobación = {auc_score:.4f}')

El indicador AUC sobre la base de comprobación = 0.9084


In [18]:
#Obtener distintos Cross Val con métricas distintas
print('Validacion Cruzada ENTRENAMIENTO (AUC):',    cross_val_score(clasif, X_train, y_train, cv=5, scoring='roc_auc'))
print('Validacion Cruzada ENTRENAMIENTO (Recall):', cross_val_score(clasif, X_train, y_train, cv=5, scoring='recall'))
print('Validacion Cruzada ENTRENAMIENTO (Precision):', cross_val_score(clasif, X_train, y_train, cv=5, scoring='precision'))
print('Validacion Cruzada ENTRENAMIENTO (F1):', cross_val_score(clasif, X_train, y_train, cv=5, scoring='f1'))

Validacion Cruzada ENTRENAMIENTO (AUC): [0.98206019 0.97670251 0.93573113 0.98584906 0.98349057]
Validacion Cruzada ENTRENAMIENTO (Recall): [0.84375    0.77419355 0.78125    0.90625    0.96875   ]
Validacion Cruzada ENTRENAMIENTO (Precision): [0.96428571 0.96       0.80645161 0.96666667 0.88571429]
Validacion Cruzada ENTRENAMIENTO (F1): [0.9        0.85714286 0.79365079 0.93548387 0.92537313]


In [19]:
f1_score(y_test, clasif.predict(X_test))

0.8807339449541285

In [20]:
y.value_counts()

,count
Tipo,
B,357
M,212


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=0)
clasif = SVC(kernel='linear', C=1, class_weight='balanced')
clasif.fit(X_train, y_train)
clasif.score(X_test, y_test)

0.8811188811188811

In [22]:
print('Validacion Cruzada (exactitud/acierto):', cross_val_score(clasif, X_train, y_train, cv=5))
print('Validacion Cruzada (AUC):',               cross_val_score(clasif, X_train, y_train, cv=5, scoring='roc_auc'))
print('Validacion Cruzada (Recall):',            cross_val_score(clasif, X_train, y_train, cv=5, scoring='recall'))

Validacion Cruzada (exactitud/acierto): [0.94186047 0.90588235 0.84705882 0.95294118 0.88235294]
Validacion Cruzada (AUC): [0.98032407 0.97491039 0.93396226 0.98290094 0.98231132]
Validacion Cruzada (Recall): [0.875      0.80645161 0.8125     0.96875    0.96875   ]


### Búsqueda GRID

In [23]:
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score

In [24]:
clasif = SVC(kernel='rbf')  # Modelo a entrenar, ya configurado
grid_val = {'gamma': [0.001, 0.0015, 0.01, 0.1, 1, 5, 10, 50]} #El hyperparametro elegido es gamma

grid_clas_ex = GridSearchCV(clasif, param_grid = grid_val, cv=3)
grid_clas_ex.fit(X_train, y_train)

print('Mejor parametro Grid (Max Exactitud): ', grid_clas_ex.best_params_)
print('Mejor score Grid (Max Exactitud): ', grid_clas_ex.best_score_)

Mejor parametro Grid (Max Exactitud):  {'gamma': 0.001}
Mejor score Grid (Max Exactitud):  0.8661971830985915


In [25]:
pd.DataFrame(grid_clas_ex.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005846,0.000964,0.003702,0.000317,0.0010,{'gamma': 0.001},0.894366,0.866197,0.838028,0.866197,0.023000,1
1,0.005450,0.000127,0.003376,0.000197,0.0015,{'gamma': 0.0015},0.880282,0.852113,0.838028,0.856808,0.017566,2
2,0.006858,0.000214,0.003945,0.000131,0.0100,{'gamma': 0.01},0.866197,0.859155,0.845070,0.856808,0.008783,2
3,0.007965,0.000774,0.005594,0.001473,0.1000,{'gamma': 0.1},0.690141,0.683099,0.647887,0.673709,0.018484,4
4,0.012378,0.001482,0.008541,0.000430,1.0000,{'gamma': 1},0.626761,0.633803,0.626761,0.629108,0.003320,5
5,0.013593,0.000937,0.008488,0.000936,5.0000,{'gamma': 5},0.626761,0.626761,0.626761,0.626761,0.000000,6
6,0.012861,0.001257,0.008129,0.000896,10.0000,{'gamma': 10},0.626761,0.626761,0.626761,0.626761,0.000000,6
7,0.011052,0.001057,0.007678,0.000336,50.0000,{'gamma': 50},0.626761,0.626761,0.626761,0.626761,0.000000,6


In [26]:
clasif = SVC(kernel='rbf')
grid_val = {'gamma': [0.001, 0.0015, 0.01,  0.1, 1, 5, 10, 50]} #

grid_clas_auc = GridSearchCV(clasif, param_grid = grid_val, scoring='roc_auc', cv=5 )
grid_clas_auc.fit(X_train, y_train)
y_decision_fn_scores_auc = grid_clas_auc.decision_function(X_test)

print('AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
print('Mejor parametro Grid (Max AUC): ', grid_clas_auc.best_params_)
print('Mejor score Grid (Max Exactitud): ', grid_clas_auc.best_score_)

AUC:  0.9528301886792453
Mejor parametro Grid (Max AUC):  {'gamma': 0.001}
Mejor score Grid (Max Exactitud):  0.9040902341020265


In [27]:
pd.DataFrame(grid_clas_auc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010821,0.001674,0.006526,0.000314,0.0010,{'gamma': 0.001},0.914931,0.907407,0.840802,0.896816,0.960495,0.904090,0.038388,1
1,0.009801,0.000585,0.007024,0.000678,0.0015,{'gamma': 0.0015},0.924190,0.898447,0.828420,0.905660,0.954009,0.902145,0.041563,2
2,0.013963,0.001078,0.008350,0.001538,0.0100,{'gamma': 0.01},0.941551,0.893070,0.786557,0.932193,0.915094,0.893693,0.056052,3
3,0.015831,0.000360,0.008599,0.000674,0.1000,{'gamma': 0.1},0.923032,0.870968,0.807783,0.875590,0.890330,0.873541,0.037593,4
4,0.016169,0.000616,0.008662,0.000300,1.0000,{'gamma': 1},0.899306,0.844683,0.791274,0.848467,0.856132,0.847972,0.034431,5
5,0.017590,0.001858,0.008638,0.000504,5.0000,{'gamma': 5},0.722801,0.707885,0.673054,0.688090,0.721403,0.702647,0.019355,6
6,0.016904,0.002161,0.009040,0.000682,10.0000,{'gamma': 10},0.634259,0.587216,0.604068,0.597288,0.602005,0.604967,0.015759,7
7,0.014105,0.000602,0.008362,0.000252,50.0000,{'gamma': 50},0.518519,0.493429,0.524764,0.490566,0.515625,0.508581,0.013888,8


In [28]:
from sklearn.metrics import get_scorer_names
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'd2_absolute_error_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_max_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples'

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [30]:
base.columns

Index(['ID', 'Tipo', 'Radio', 'Textura', 'Perimetro', 'Area', 'Suavidad',
       'Compactavidad', 'Concavidad', 'Puntos_concavos', 'Simetria',
       'Fractal'],
      dtype='object')

In [31]:
from sklearn.model_selection import GridSearchCV

X = base[['Radio', 'Simetria']]
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=0)
clasif = SVC(kernel = 'linear').fit(X_train, y_train)
grid_val = {'class_weight':['balanced', {1:1},{1:3},{1:4}], 'gamma': [0.001, 0.01, 0.1, 1, 5, 10, 50]}

for i, eval_metric in enumerate(('precision', 'recall', 'f1', 'roc_auc')):
    grid_clas_p = GridSearchCV(clasif, param_grid=grid_val, scoring=eval_metric, cv=3)
    grid_clas_p.fit(X_train, y_train)
    print('Mejor Parametro Grid (max, {0}): {1}'.format(eval_metric, grid_clas_p.best_params_))
    print('Mejor Score Grid (max, {0}): {1}'.format(eval_metric, grid_clas_p.best_score_))

Mejor Parametro Grid (max, precision): {'class_weight': {1: 1}, 'gamma': 0.001}
Mejor Score Grid (max, precision): 0.9230404823428079
Mejor Parametro Grid (max, recall): {'class_weight': {1: 4}, 'gamma': 0.001}
Mejor Score Grid (max, recall): 0.8930817610062892
Mejor Parametro Grid (max, f1): {'class_weight': {1: 1}, 'gamma': 0.001}
Mejor Score Grid (max, f1): 0.822288055006949
Mejor Parametro Grid (max, roc_auc): {'class_weight': {1: 4}, 'gamma': 0.001}
Mejor Score Grid (max, roc_auc): 0.9369655854709915


### Implementación de Pipelines

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import time

In [33]:
modelo = Pipeline([('scaler', MinMaxScaler()), ('model', SVC())])
param_grid = {'model__gamma': [0.001, 0.01, 0.1, 1, 5, 10, 50], 'model__kernel':  ['rbf', 'linear'] }
cv = GridSearchCV(modelo, param_grid, scoring='f1', cv=5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', SVC())]),
             param_grid={'model__gamma': [0.001, 0.01, 0.1, 1, 5, 10, 50],
                         'model__kernel': ['rbf', 'linear']},
             scoring='f1')

In [34]:
cv.best_params_

{'model__gamma': 50, 'model__kernel': 'rbf'}

In [35]:
cv.score(X_test, y_test)

0.8155339805825242

### EJERCICIO
REALICE UN GRID SEARCH CON CROSS VALIDATION, UTILICE KERNEL LINEAL Y RBF, PRUEBE LOS PARAMETROS C{0.1 , 10} y un cv=5

In [38]:
modelo = Pipeline([('scaler', MinMaxScaler()), ('model', SVC())])
param_grid = {'model__gamma': [0.1, 10], 'model__kernel':  ['rbf', 'linear'] }
cv = GridSearchCV(modelo, param_grid, scoring='precision', cv=5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', SVC())]),
             param_grid={'model__gamma': [0.1, 10],
                         'model__kernel': ['rbf', 'linear']},
             scoring='precision')